# Notes from "VQE Zero to Hero" video

Video by Qiskit. [Here](https://www.youtube.com/watch?v=TUFovZsBcW4)

Very similar to this paper: [Use VQE to calculate the ground energy of hydrogen molecules on IBM Quantum](https://arxiv.org/pdf/2305.06538)

## The idea

Finding ground state energy of a molecule.

1. Formalize using time independent Schrodinger equation.

Hamiltonian $\hat H$ (that describes the system/molecules) acting on wavefunction $|\psi\rangle$, which yields a specific energy (numeric value) $E$:

$$
\hat H |\psi\rangle = E|\psi\rangle
$$

(i.e. this is an eigenvalue problem - what is the smallest?)

2. Expval is bounded by this ground state energy:

$$
\langle \psi | \hat H | \psi \rangle \geq E_0
$$

So we want to MINIMIZE expval, to approximate $E_0$.

3. Parameterizing wave function (with angle $\theta$)

Find 

$$
\text{min}_\theta \langle \Psi(\theta)|\hat H | \Psi(\theta)\rangle
$$


## Formalizing (mapping) for computer problem

Hamiltonian $\hat H$ describes energy of the system: 

$$
H = \hat T + \hat V
$$,

where potential energy $\hat V$ depends on position $r$,

kinetic energy $\hat T = \frac{\hbar^2}{2m} \nabla^2$ ($\nabla^2$ is Laplacian, or gradient of divergence) (typical expansion of Schrodinger equation).

And now apply this for molecules.

1. Potential energy

$\hat V = \sum^e_{i,j}\frac{e^2}{4\pi\varepsilon_0|r_i-r_j|} + \sum^n_{i, j}\frac{Z_i Z_j e^2}{4\pi\varepsilon_0|r_i-r_j|} - \sum^e_i\sum^n_j\frac{Z_j e^2}{4\pi\varepsilon_0|r_i-r_j|}$

Sources: electrons and nuclii. Coulomb interactions: e-e, N-N, e-N.

2. Kinetic energy

$\hat T = -\sum^n_i\frac{\hbar^2}{2m_i}\nabla^2_i - \sum^e_i\frac{\hbar^2}{2m_e}\nabla^2_i$

From nuclii and electrons.

3. Simplify

Using Bohr-Oppenheimer approximation (assumes that nuclii are stationary points). 

$T_{nuclii} = 0$ and $V_{nuclii} = constant$

Now purely electronic Hamiltonian (3 terms).

### Classical computing representation (first quantization)

And now let's look at the wave function.

Representing it (in classical computing sense) as Slater determinant (of matrix with electron orbitals $\chi$ with position and spin info $x$):

> Remember Pauli-exclusion principle

$\Psi = \frac{1}{\sqrt{N!}}  \left|\begin{matrix} \chi_1(x_1) & \dots & \chi_1(x_n) \\ & \ddots & \\ & & \chi_n(x_n) \end{matrix}\right|$

Matrix anti-symmetric under exchange(?)...

Sometimes in this format (list of orbitals that are filled): $|\chi_1 \dots \chi_n \rangle$

### Quantum computing representation (second quantiziation)

Occupation number formalizm.

$\Psi = \sum c_i |n_1 \dots n_m\rangle$

Here $n_j$ are binary occupation numbers (is electron in *that* state).

To operate on this occupation number formalizm, we use **fermionic creation and anialation operators**: $\hat a ^\dagger _i$ and $\hat a _j$.

These will flip 0s and 1s, also will have some sign changes... - create or remove fermions from the specific positions ($i$ and $j$).

So now (dropping 4 pi epsilon - unitization(?))

$\hat H \approx -\sum^e_i \frac 12 \nabla^2_i + \sum ^e_{i,j} \frac{1}{|r_i - r_j|} - \sum^e_i \sum^n_j \frac{Z_j}{|r_i - r_j|}$

- reduced electronic hamiltonian.

For second quantization - rules to convert/map operators. Single particle operators:

$$
\sum_i h(x_i) \Rightarrow \sum _{pq} \langle p | \hat h | q \rangle \hat a^\dagger _p \hat a _q
$$

For that we can rewrite as (constant):

$$
h_{pq} = \int_{-\infty}^{+\infty} \Psi_p(x) (\frac 12 \nabla^2_i - \sum^n_j \frac{Z_j}{|r_i - r_j|}) \Psi_q(x) dx = c \cdot \hat a^\dagger_p \hat a_q
$$

Another integral would be for repulsion:

$$
\sum _{i,j} h(x_i, x_j) \Rightarrow \sum_{p,q,r,s} \langle qr|\hat h | rs \rangle \hat a ^\dagger _p \hat a^\dagger_q \hat a_r \hat a_s
$$

Constant:
$$
h_{pqrs} = \int \int \Psi^\star_p(x_1)\Psi^\star (x_1) (\frac {1}{|r_1 - r_2|}) \Psi_r(x_2)\Psi_s(x_2)|\chi_1 \dots \chi_n\rangle
$$

And adding all this up - second quantized hamiltonian form...

$$
H = h_{pq}\hat a^\dag_p \hat a_q + h_{pqrs}\hat a^\dag_p \hat a^\dag_q \hat a_r \hat a_s + h_v
$$

### Operator mapping to quantum computing operators

Now we just need to convert fermionic creation and anialation operators. From there everything is straight forward.

Techniques to convert these operators: Parady mapping, Jordan-Vignor, ...

e.g. common in literature Jordan-Vignor (anialation at position p maps to Pauli operators):

$$
\hat a_p \rightarrow \frac 12 (X_p + iY_p)Z_i \dots Z_{p-1}
$$

Apply molecular hamiltonian to get expval. We have parameterized $|\Psi(\theta)\rangle$.

Final process: get the output of the parameterized gates, apply this hamiltonian and repeat this to minimize parameters.

> Omg I'm gonna die...
